In [ ]:
#
# Simple example of a Julia linear pogramming model
#
# Convert from notebook to julia program:
#   python3 nb_to_jl.py lp_claude_revised_simplex.ipynb
#
# Run the program:
#   julia lp_llama_ipm.jl -f ../benchmarks/mps_files/ex97.mps -v --no_presolve
#
 
using LinearAlgebra
using SparseArrays
using Random
using ArgParse

# local modules
push!(LOAD_PATH, realpath("../src"))
using lp_constants
using lp_utils
using lp_problem
using lp_read_mps
using lp_standard_form_converter

if is_running_in_notebook()
    println("Running as a notebook - using local options")
    const DEFAULT_USE_SIMPLEX_METHOD = true
    const DEFAULT_USE_INTERIOR_POINT_METHOD = false
    const DEFAULT_USE_NO_PRESOLVE = false 
    const DEFAULT_MINIMIZE_OBJECTIVE = true
    const DEFAULT_MAXIMIZE_OBJECTIVE = false
    const DEFAULT_VERBOSE = true    
else
    #
    # do NOT change these values here - use command line options
    #

    println("Running as a script - using command line options")
    const DEFAULT_USE_SIMPLEX_METHOD = true
    const DEFAULT_USE_INTERIOR_POINT_METHOD = false
    const DEFAULT_USE_NO_PRESOLVE = true 
    const DEFAULT_MINIMIZE_OBJECTIVE = true
    const DEFAULT_MAXIMIZE_OBJECTIVE = false
    const DEFAULT_VERBOSE = false    
end

global options = Dict()

const MPS_EXAMPLE_FILENAME = "../check/problems/mps_files/ex_9-7.mps"
# const MPS_EXAMPLE_FILENAME::String = "../benchmarks/mps_files/ex4-3.mps"

In [ ]:
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--filename", "-f"
            help = "path to the problem file (mps format)"
            arg_type = String
            required = true
        "--interior", "-i"
            help = "use interior point method (LP only)"
            action = :store_true
        "--min"
            help = "minimization of the objective function (default)"
            action = :store_true            
        "--max"
            help = "maximization of the objective function"
            action = :store_true            
        "--no_presolve"
            help = "do not presolve (default false)"
            action = :store_true
        "--simplex", "-s"
            help = "use simplex method (default)"
            action = :store_true
        "--verbose", "-v"
            help = "verbose output"
            action = :store_true
    end

    return parse_args(s)
end

In [ ]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, lp.A, lp.b
    l, u = copy(lp.l), copy(lp.u)  # Create copies to avoid modifying the original bounds
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] == A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] == -A[j, :] && abs(b[i] + b[j]) < eps &&
                    ((constraint_types[i] == 'L' && constraint_types[j] == 'G') ||
                     (constraint_types[i] == 'G' && constraint_types[j] == 'L'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4 - Conservative bound tightening with feasibility checks - k is the selected column
    bound_updates = true
    iteration = 1
    while bound_updates
        bound_updates = false
        println("\nBound tightening loop: iteration $iteration")

        lower_bounds = [Float64[] for _ in 1:n]
        upper_bounds = [Float64[] for _ in 1:n]  

        for i in 1:m  # Loop over constraints
            if !keep_rows[i]
                continue
            end

            println("\nAnalyzing bounds for constraint $i:")
            row = A[i, :]  # Get the i-th row            

            for k in 1:n  # select variable column
                if (A[i,k] == 0) || !keep_cols[k]
                    continue
                end

                lower_sum = sum((l[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij > 0), init=0.0)
                upper_sum = sum((u[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij < 0), init=0.0)
                new_bound = (b[i] - lower_sum - upper_sum)/ A[i,k]
                println("  analyzing bounds for variable $(vars[k]): k = $(k): A[i,k] = $(A[i,k]), b[i] = $(b[i]), lower_sum = $lower_sum, upper_sum = $upper_sum, new_bound = $new_bound")
                
                if constraint_types[i] == 'L'
                    if A[i,k] > 0
                        push!(upper_bounds[k], new_bound)
                    else
                        push!(lower_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'G'
                    if A[i,k] > 0
                        push!(lower_bounds[k], new_bound)
                    else
                        push!(upper_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'E'
                    push!(lower_bounds[k], new_bound)
                    push!(upper_bounds[k], new_bound)
                end

                if !isempty(lower_bounds[k])
                    if length(lower_bounds[k]) > 0
                        new_lower = maximum(lower_bounds[k])
                        if new_lower > l[k] + eps
                            println("variable $(vars[k]) tightening lower bound: $new_lower (prev: $(l[k]))")
                            l[k] = new_lower
                            bound_updates = true                    
                        end
                    end
                end
                
                if !isempty(upper_bounds)
                    if length(upper_bounds[k]) > 0
                        new_upper = minimum(upper_bounds[k])
                        if new_upper < u[k] - eps
                            println("variable $(vars[k]) tightening upper bound: $new_upper (prev: $(u[k]))")
                            u[k] = new_upper
                            bound_updates = true
                        end
                    end
                end
                
                # Ensure lower bound doesn't exceed upper bound
                if l[k] > u[k] + eps
                    println("WARNING: variable $(vars[k]) lower bound ($(l[k])) exceeds upper bound ($(u[k])) for $(vars[k]) - adjusting bounds to maintain feasibility")
                    avg_bound = (l[k] + u[k]) / 2
                    l[k] = avg_bound - eps
                    u[k] = avg_bound + eps
                end
                println("calculated bounds for $(vars[k]): [$(l[k]), $(u[k])]")                
            end
        end
           
        if bound_updates
            iteration += 1
        else
            println("No further bound updates possible")
        end
    end

    # Step 5: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) == 1 && constraint_types[i] == 'E'
                val = b[i] / col[i]
                if l[j] <= val && val <= u[j]
                    fixed_vars[vars[j]] = val
                    keep_cols[j] = false
                    b .-= val * col
                end
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    l_new = l[keep_cols]
    u_new = u[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = 0.0
    for (j, var) in enumerate(vars)
        if haskey(fixed_vars, var)
            obj_adjust += c[j] * fixed_vars[var]
        end
    end

    if options[OPTION_VERBOSE]
        println("\nPresolve summary:")
        println("  Original problem size: $(m) x $(n)")
        println("  Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("  Number of fixed variables: $(length(fixed_vars))")
        println("  Objective adjustment: $obj_adjust")
        println("  Updated bounds:")
        for (var, lb, ub) in zip(vars_new, l_new, u_new)
            println("    $var: [$lb, $ub]")
        end
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, l_new, u_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.

In [ ]:
function revised_simplex(lp::LPProblem)
    println("Converting problem to standard form...")
    A, b, c = convert_to_standard_form(lp)
    m, n = size(A)
    
    println("\nStandard form problem:")
    println("  Objective function coefficients c: ", c)
    println("  Constraint matrix A: ", A)
    println("  Right-hand side b: ", b)
    println("  Variables: ", [lp.vars; ["s$i" for i in 1:(n-length(lp.vars))]])
    println("  Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    println("  constraint_types = $(lp.constraint_types)")
    
    # Initialize basis with slack variables
    B = collect((length(lp.vars)+1):n)
    N = collect(1:length(lp.vars))
    
    println("\nInitial basis: ", B)
    println("Initial non-basic variables: ", N)

    # Initialize B_factor outside the loop
    B_matrix = A[:, B]
    B_factor = lu(B_matrix)    
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]
        x_B = B_matrix \ b
        println("  Basic solution x_B: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'
        c_N = c[N] - A[:, N]' * y
        println("  Dual variables y: ", y)
        println("  Reduced costs c_N: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .>= -1e-10)
            x = zeros(n)
            x[B] = x_B
            println("\nOptimal solution found:")
            obj_value = dot(lp.is_minimize ? c : -c, x)
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable
        e = argmin(c_N)
        q = N[e]
        println("  Entering variable: ", q)
        
        # Step 5: Compute direction
        aq = A[:, q]  # Extract q-th column of A
        d = Vector(B_factor \ Vector(aq))  # Convert to dense, solve, and convert back
        println("  Direction d: ", d)
        
        # Step 6 & 7: Check unboundedness and choose leaving variable
        if all(d .<= 1e-10)
            error("Problem is unbounded")
        end
        
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf
        valid_ratios = filter(x -> x > 0, ratios)
        if isempty(valid_ratios)
            error("Problem is unbounded")
        end
        l = argmin(valid_ratios)
        p = B[l]
        println("  Leaving variable: ", p)

        # Step 8: Update basis
        B[l] = q
        N[e] = p
        println("  New basis: ", B)
        println("  New non-basic variables: ", N)
        
        # Update B_factor when the basis changes
        B_matrix = A[:, B]
        B_factor = lu(B_matrix)
        
        if iteration > 10  # FIXME: Add a better termination criterion
            error("Maximum iterations reached")
        end
    end
end

In [ ]:
function corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
    println("Computing corrector direction:")
    
    # Get dimensions
    m, n = size(A)
    println("  m: $m, n: $n")

    # Dimension checks
    if size(A, 1) != m || size(A, 2) != n
        error("A should be an m×n matrix. Got size(A) = $(size(A))")
    end
    if length(r_b) != m || length(r_s) != m || length(s) != m || length(Δy_aff) != m
        error("r_b, r_s, s, and Δy_aff should be m-dimensional vectors. 
               Got lengths: r_b ($(length(r_b))), r_s ($(length(r_s))), 
               s ($(length(s))), Δy_aff ($(length(Δy_aff)))")
    end
    if length(x) != n || length(Δx_aff) != n
        error("x and Δx_aff should be n-dimensional vectors. 
               Got lengths: x ($(length(x))), Δx_aff ($(length(Δx_aff)))")
    end

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    Δx_aff = vec(Δx_aff)
    Δy_aff = vec(Δy_aff)

    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")
    println("  Δx_aff: $Δx_aff")
    println("  Δy_aff: $Δy_aff")

    # Compute Δs_aff
    Δs_aff = r_s - A * Δx_aff

    # Compute μ (barrier parameter)
    μ = (dot(x, A' * s) / n) * ((s + Δs_aff)' * (A * (x + Δx_aff)) / (s' * (A * x)))^3

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b - A * Δx_aff;
           A' * r_s - A' * Δs_aff - (x .* (A' * Δs_aff) + A' * s .* Δx_aff - μ * ones(n))]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy_cor = direction[1:m]
    Δx_cor = direction[m+1:end]

    return Δx_cor, Δy_cor
end

In [ ]:
function affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
    println("Computing affine scaling direction:")
    println("  A: $A")
    println("  r_b: $r_b")
    println("  r_s: $r_s")
    println("  x: $x")
    println("  s: $s")

    m, n = size(A)
    println("  m: $m, n: $n")

    # Ensure vectors are column vectors
    r_b = vec(r_b)
    r_s = vec(r_s)
    x = vec(x)
    s = vec(s)
    y = vec(y)
    r_c = vec(r_c)

    # Compute K matrix
    K = [spdiagm(0 => s) A; A' -spdiagm(0 => x)]
    println("  K: $K")

    # Compute rhs vector
    rhs = [r_b; -(A' * y + r_c)]
    println("  rhs: $rhs")

    # Solve system
    direction = K \ rhs
    println("  direction: $direction")

    # Extract and return the components of the direction
    Δy = direction[1:m]
    Δx = direction[m+1:end]

    return Δx, Δy
end

In [ ]:
function compute_step_length(v, Δv)
    α = 1.0
    for i in eachindex(v)
        if Δv[i] < 0
            α = min(α, -v[i] / Δv[i])
        end
    end
    return α
end

In [ ]:
#=
The rest of the Interior Point Method implementation looks correct. The main loop in the interior_point_method function follows the standard steps:

Compute residuals
Compute affine scaling direction
Compute corrector direction
Combine directions and update variables
Update barrier parameter
Check for convergence

The method also includes proper logging and debugging output, which is helpful for understanding the algorithm's progress.
To further improve the code, you might consider:

Adding error handling for cases where the problem doesn't converge within the maximum number of iterations.
Implementing a dynamic update strategy for the barrier parameter μ.
Adding more sophisticated step length calculations to ensure variables remain positive.
=#


function interior_point_method(lp::LPProblem; 
                               tolerance=1e-8, 
                               max_iterations=1000, 
                               barrier_parameter=0.1)
    # Initialize variables
    n = length(lp.vars)
    m = size(lp.A, 1)
    x = ones(n)
    s = ones(m)
    y = zeros(m)
    μ = barrier_parameter
    iteration = 0

    println("Starting IPM solver with parameters:")
    println("  Tolerance: $tolerance")
    println("  Max iterations: $max_iterations")
    println("  Barrier parameter: $barrier_parameter")

    # Main loop
    while iteration < max_iterations
        println("\nIteration $iteration:")

        # Compute residuals
        r_c = lp.c - A' * y
        r_b = b - A * x
        r_s = s - (A * x - b)
        println("  Residuals:")
        println("    r_c: $r_c")
        println("    r_b: $r_b")
        println("    r_s: $r_s")

        # Compute affine scaling direction
        Δx_aff, Δy_aff = affine_scaling_direction(A, r_b, r_s, x, s, y, r_c)
        Δs_aff = r_s - A * Δx_aff
        println("  Affine scaling direction:")
        println("    Δx_aff: $Δx_aff")
        println("    Δs_aff: $Δs_aff")
        println("    Δy_aff: $Δy_aff")

        # Compute corrector direction
        Δx_cor, Δy_cor = corrector_direction(A, r_b, r_s, x, s, Δx_aff, Δy_aff)
        Δs_cor = r_s - A * Δx_cor
        println("  Corrector direction:")
        println("    Δx_cor: $Δx_cor")
        println("    Δs_cor: $Δs_cor")
        println("    Δy_cor: $Δy_cor")

        # Combine directions and update variables
        Δx = Δx_aff + Δx_cor
        Δs = Δs_aff + Δs_cor
        Δy = Δy_aff + Δy_cor

        # Compute step length
        α_pri = compute_step_length(x, Δx)
        α_dual = compute_step_length(s, Δs)
        α = min(0.99 * min(α_pri, α_dual), 1)

        x += α * Δx
        s += α * Δs
        y += α * Δy
        println("  Updated variables:")
        println("    x: $x")
        println("    s: $s")
        println("    y: $y")

        # Update barrier parameter
        μ = (dot(x, A' * s)) / n
        println("  Updated barrier parameter: $μ")

        # Check convergence
        if norm([r_c; r_b; r_s]) < tolerance && μ < tolerance
            println("Converged in $iteration iterations")
            break
        end

        iteration += 1
    end

    return x, y, iteration
end

In [ ]:
# main program

options[OPTION_USE_SIMPLEX_METHOD] = DEFAULT_USE_SIMPLEX_METHOD
options[OPTION_USE_NO_PRESOLVE] = DEFAULT_USE_NO_PRESOLVE
options[OPTION_USE_INTERIOR_POINT_METHOD] = DEFAULT_USE_INTERIOR_POINT_METHOD
options[OPTION_MINIMIZE_OBJECTIVE] = DEFAULT_MINIMIZE_OBJECTIVE
options[OPTION_MAXIMIZE_OBJECTIVE] = DEFAULT_MAXIMIZE_OBJECTIVE
options[OPTION_VERBOSE] = DEFAULT_VERBOSE

mps_filename = MPS_EXAMPLE_FILENAME
if !is_running_in_notebook()
    parsed_args = parse_commandline()
    # println("Parsed args: $parsed_args")

    # Process command-line name
    if haskey(parsed_args, COMMAND_LINE_OPTION_FILENAME) && !isnothing(parsed_args[COMMAND_LINE_OPTION_FILENAME])
        mps_filename = parsed_args[COMMAND_LINE_OPTION_FILENAME]
        if !isfile(mps_filename)       
            error("File not found or could not read: '$mps_filename'")
        end
    end

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD])
        options[OPTION_USE_SIMPLEX_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_SIMPLEX_METHOD]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end


    if haskey(parsed_args, COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE])
        options[OPTION_MINIMIZE_OBJECTIVE] = parsed_args[COMMAND_LINE_OPTION_MINIMIZE_OBJECTIVE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE])
        options[OPTION_MAXIMIZE_OBJECTIVE] = parsed_args[COMMAND_LINE_OPTION_MAXIMIZE_OBJECTIVE]
    end

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_NO_PRESOLVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_NO_PRESOLVE])
        options[OPTION_USE_NO_PRESOLVE] = parsed_args[COMMAND_LINE_OPTION_USE_NO_PRESOLVE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_VERBOSE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_VERBOSE])
        options[OPTION_VERBOSE] = parsed_args[COMMAND_LINE_OPTION_VERBOSE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end
end

# set defaults if not explicably set
if !options[OPTION_USE_SIMPLEX_METHOD] && !options[OPTION_USE_INTERIOR_POINT_METHOD]
    options[OPTION_USE_SIMPLEX_METHOD] = true
elseif options[OPTION_USE_SIMPLEX_METHOD] && options[OPTION_USE_INTERIOR_POINT_METHOD]
    error("Cannot use both simplex and interior point methods")
end

if !options[OPTION_MINIMIZE_OBJECTIVE] && !options[OPTION_MAXIMIZE_OBJECTIVE]
    options[OPTION_MINIMIZE_OBJECTIVE] = true
elseif options[OPTION_MINIMIZE_OBJECTIVE] && options[OPTION_MAXIMIZE_OBJECTIVE]
    error("Cannot maximize and minimize the objective at the same time")
end

if options[OPTION_VERBOSE]
    println("Options:")
    println("  Use no presolve: $(options[OPTION_USE_NO_PRESOLVE])")
    println("  Use simplex method: $(options[OPTION_USE_SIMPLEX_METHOD])")
    println("  Use interior point method: $(options[OPTION_USE_INTERIOR_POINT_METHOD])")
    println("  Objective: $(options[OPTION_MINIMIZE_OBJECTIVE] ? OPTION_MINIMIZE_OBJECTIVE : OPTION_MAXIMIZE_OBJECTIVE)")
    println("  Verbose output: $(options[OPTION_VERBOSE])")
end

# read the mps file
lp = read_mps_from_file(mps_filename)

if options[OPTION_VERBOSE]
    println("\nOriginal problem:")
    println("  Minimize: $(lp.is_minimize)")
    println("  c = $(lp.c)")
    println("  A = $(lp.A)")
    println("  b = $(lp.b)")
    println("  l = $(lp.l)")
    println("  u = $(lp.u)")        
    println("  vars = $(lp.vars)")
    println("  constraint_types = $(lp.constraint_types)")
end

# Presolve the problem
if !options[OPTION_USE_NO_PRESOLVE]
    println("\nPresolving the problem")
    lp, fixed_vars, obj_adjust = presolve(lp)

    println("\nFixed variables:")
    for (var, val) in fixed_vars
        println("$var = $val")
    end

    println("\nReduced problem:")
    println("  Minimize: $(lp.is_minimize)")
    println("  c = $(lp.c)")
    println("  A = $(lp.A)")
    println("  b = $(lp.b)")
    println("  l = $(lp.l)")
    println("  u = $(lp.u)")     
    println("  vars = $(lp.vars)")
    println("  constraint_types = $(lp.constraint_types)")
end

# solve the LP problem

if options[OPTION_USE_INTERIOR_POINT_METHOD]
        println("\nSolving the problem using the Interior Point Method")
        execution_time = @elapsed x, y, iterations = interior_point_method(lp) 
else
    # if options[OPTION_USE_SPARSE_MATRIX]
    #     println("\nSolving the problem using the Revised Simplex Method with sparse matrices")
    #     execution_time = @elapsed x, obj_value = revised_simplex_sparse(lp)
    # else
        println("\nSolving the problem using the Revised Simplex Method with dense matrices")
        execution_time = @elapsed x, obj_value = revised_simplex(lp)
    # end
end
println("\nExecution time: $execution_time seconds")

# show the solution

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")